# Extract From HKTV MALL

使用selenium從hktvmall取得產品目錄 (Beautifulsoup 無法取得 ajax 架構的資料)

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
import time

In [2]:
# selenium init (firefox)
indexPage = webdriver.Firefox()
url = "https://www.hktvmall.com/hktv/zh/main/%E7%B6%AD%E7%89%B9%E5%81%A5%E9%9D%88/s/H0303001?q=::storeH0303001:"
indexPage.implicitly_wait(20)
indexPage.get(url)
time.sleep(2)

In [3]:
# RemoveAds
try: 
    indexPage.execute_script('document.getElementsByClassName("CrazyAd")[0].remove();')
except:
    1

# Grab Product
productDiv = indexPage.find_elements(By.CLASS_NAME, "product-brief")

In [4]:
# Grab Product titles, Prices and links
titles = []
prices = []
links = []
for product in productDiv:
    titles.append(product.find_element(By.CLASS_NAME, "brand-product-name").text)
    prices.append(product.find_element(By.CLASS_NAME, "price").text)
    links.append(product.find_element(By.CSS_SELECTOR, '[href^=main]').get_attribute('href'))

In [5]:
# Write into productDF
productDF = pd.DataFrame({"ProductName":titles, "Price":prices, "Link":links, "AvgStars":None, "Star5":None, "Star4":None, "Star3":None, "Star2":None, "Star1":None})

In [6]:
# Remove non product data
productDF = productDF.drop(productDF[(productDF['ProductName']=='') & (productDF['Price']=='')].index).reset_index(drop=1)

進入產品URL, 取得用戶評價

In [7]:
reviews = pd.DataFrame({"Link":[], "Comment":[], "Stars":[]})
for i, link in enumerate(productDF['Link']):
    try:
        # init selenium (firefox)
        productPage = webdriver.Firefox()
        productPage.implicitly_wait(20)
        productPage.get(link+"?scrollTo=reviewTab")
        time.sleep(2)

        # RemoveAds
        try: 
            productPage.execute_script('document.getElementsByClassName("CrazyAd")[0].remove();')
        except:
            1

        # Append ProductDF stars
        productDF["AvgStars"].loc[i] = productPage.find_element(By.CSS_SELECTOR, 'label.title-text').text
        productDF["Star5"].loc[i] = productPage.find_element(By.CSS_SELECTOR, 'div[data-tabname="star5"]').text
        productDF["Star4"].loc[i] = productPage.find_element(By.CSS_SELECTOR, 'div[data-tabname="star4"]').text
        productDF["Star3"].loc[i] = productPage.find_element(By.CSS_SELECTOR, 'div[data-tabname="star3"]').text
        productDF["Star2"].loc[i] = productPage.find_element(By.CSS_SELECTOR, 'div[data-tabname="star2"]').text
        productDF["Star1"].loc[i] = productPage.find_element(By.CSS_SELECTOR, 'div[data-tabname="star1"]').text

        for star in range(5,0,-1):
            comments = []            
            try:
                # Simulate User click on the [5~1]stars reviews
                productPage.find_element(By.CSS_SELECTOR, f'div[data-tabname="star{star}"]').click()
                
                # Grab how many comments are there
                reviewNo = re.findall(r"\(\d+\)", productPage.find_element(By.CSS_SELECTOR, f'div[data-tabname="star{star}"]').text)[0]
                reviewNo = re.sub(r'\(', '', reviewNo)
                reviewNo = re.sub(r'\)', '', reviewNo)
                reviewNo = int(reviewNo)

                # Grab first page comments
                for reviewi in productPage.find_elements(By.CSS_SELECTOR, 'div.review-title'):
                    comments.append(reviewi.text)

                # Since 1 page show 10 data, if the data%10==0 do 1 less click on "next page"
                if (reviewNo%10)==0:
                    reviewNo -= 1

                # Grab next pages comments
                for pageNo in range(int(reviewNo/10)):
                    productPage.find_element(By.CSS_SELECTOR, 'a.next-btn').click()
                    for commenti in productPage.find_elements(By.CSS_SELECTOR, 'div.review-title'):
                        comments.append(commenti.text)
                reviews = pd.concat([reviews, pd.DataFrame({"Link":link, "Comment":comments, "Stars":star})])
            except:
                # No comment on this star
                1
        # close firefox
        productPage.close()
    except:
        print(f'{link} No Comment or Connection Error')
        # close firefox
        productPage.close()

https://www.hktvmall.com/hktv/zh/main/%E7%B6%AD%E7%89%B9%E5%81%A5%E9%9D%88/s/main/%E7%B6%AD%E7%89%B9%E5%81%A5%E9%9D%88/s/H0303001/%E8%AD%B7%E7%90%86%E4%BF%9D%E5%81%A5/%E8%AD%B7%E7%90%86%E4%BF%9D%E5%81%A5/%E4%BF%9D%E5%81%A5%E7%94%A2%E5%93%81/%E7%BE%8E%E8%82%8C%E9%A3%9F%E5%93%81/%E6%97%A5%E5%A4%9C%E7%BE%8E%E7%99%BD%E4%B8%B8-30%E7%B2%92-x-2%E7%9B%92%E9%99%A4%E6%96%91%E6%B7%A1%E6%96%91%E7%BE%8E%E7%99%BD%E6%B8%9B%E5%B0%91%E9%BB%91%E8%89%B2%E7%B4%A0%E6%B2%88%E6%BE%B1/p/H0303001_S_VDPWCA030BXHK01_B No Comment or Connection Error
https://www.hktvmall.com/hktv/zh/main/%E7%B6%AD%E7%89%B9%E5%81%A5%E9%9D%88/s/main/%E7%B6%AD%E7%89%B9%E5%81%A5%E9%9D%88/s/H0303001/%E8%AD%B7%E7%90%86%E4%BF%9D%E5%81%A5/%E8%AD%B7%E7%90%86%E4%BF%9D%E5%81%A5/%E4%BF%9D%E5%81%A5%E7%94%A2%E5%93%81/%E5%85%8D%E7%96%AB%E7%B3%BB%E7%B5%B1/%E7%9B%88%E6%B4%BB%E9%9B%B2%E8%8A%9D%E7%B3%BB%E5%88%97%E7%99%8C%E7%97%87%E6%B2%BB%E7%99%82%E5%B0%88%E7%94%A8%E7%87%9F%E9%A4%8A%E5%93%81%E6%8A%97%E7%82%8E%E5%A2%9E%E5%BC%B7%E5%85%8D%E7%96%AB%E5%8

In [8]:
# close firefox
indexPage.close()

#### Transfer to CSV

In [9]:
# productDF.to_csv('products.csv')

In [10]:
# reviews.to_csv('comment.csv')

# Transform

#### Basic data transform of productDF and reviews

In [11]:
def stringInBracket(x):
    if (pd.isna(x)) | (x=='') :
        y = 0
    else:
        y = re.findall(r'\(\d+\)', x)[0]
        y = re.sub(r'\(', '', y)
        y = re.sub(r'\)', '', y)
        y = int(y)
    return y
for i in range(1,6):
    productDF[f'Star{i}'] = productDF[f'Star{i}'].apply(lambda x: stringInBracket(x))

In [12]:
productDF["AvgStars"] = productDF["AvgStars"].str.replace('/5', '')

In [13]:
productDF["AvgStars"] = productDF["AvgStars"].fillna(0).astype(float)

In [14]:
productDF['Price'] = productDF['Price'].str.replace(r'$ ', "")
productDF['Price'] = productDF['Price'].str.replace(r',', "")
productDF['Price'] = productDF['Price'].astype(float)

In [15]:
reviews['Stars'] = reviews['Stars'].astype(int)
reviews = reviews.reset_index(drop=1)

In [16]:
reviews['Comment'] = reviews['Comment'].fillna('')

#### Add keywordsDF

In [17]:
# 中文keywords modules
import jieba
import jieba.analyse
from collections import Counter
dictfile = "./src/dict.txt"  # 字典檔
stopfile = "./src/stopwords.txt"  # stopwords
jieba.set_dictionary(dictfile)
jieba.analyse.set_stop_words(stopfile)

In [18]:
# 將所有comments混合成單一string
allcomments = ""
for text in reviews['Comment']:
     allcomments+=text

# 取600個keywords
tags = jieba.analyse.extract_tags(allcomments, topK=600) 

freq = {}
# 計算 frequency
seg_list = jieba.lcut(allcomments, cut_all=False)
dictionary = Counter(seg_list)
for ele in dictionary:
    if ele in tags:
        freq[ele] = dictionary[ele]

Building prefix dict from c:\Users\user\Code\vitagreen\dict.txt ...
Dumping model to file cache C:\Users\user\AppData\Local\Temp\jieba.u4d3b91421de6d129127e9163cdb4a990.cache
Loading model cost 0.602 seconds.
Prefix dict has been built successfully.


In [19]:
#寫入 keywordsDF
keywordsDF = pd.DataFrame()
for kw, ct in freq.items():
    keywordsDF = pd.concat([keywordsDF, pd.DataFrame({'keywords':[kw], 'freq':[ct]})])

In [20]:
keywordsDF = keywordsDF.set_index('keywords')

In [21]:
reviews['keywords'] = ''
for keyword in keywordsDF.index:
    # print(keyword)
    for i in reviews[reviews['Comment'].str.contains(keyword)].index:
        reviews.loc[i, 'keywords'] += f"{keyword}, "
keywordsIncludedDF = pd.DataFrame({'included_keywords':reviews['keywords']})

#### Transfer to CSV

In [22]:
# keywordsDF.to_csv('keywords.csv')

In [23]:
# keywordsIncludedDF.to_csv('commentIncludeKeywords')

# Load to SQL

In [44]:
from sqlalchemy import create_engine
import constraint # Save your sql connection data into constraint.py
user = constraint.user
password = constraint.password
hostname = constraint.hostname
database_name = constraint.database_name
schema_name = 'vitagreen'
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{hostname}/{database_name}")

#### productDF to postgreSQL("vitagreen"."Products")

In [45]:
# productDF.to_csv('productDF_transformed.csv')
connection = engine.raw_connection()
cursor = connection.cursor()
for index, row in productDF.iterrows():
     sqlMsg = f'INSERT INTO "vitagreen"."Products" (index, product_name, price, link, avg_stars, stars5, stars4, stars3, stars2, stars1) \
          VALUES ({index}, \'{row.ProductName}\', {row.Price}, \'{row.Link}\', {row.AvgStars}, {row.Star5}, {row.Star4}, {row.Star3}, {row.Star2}, {row.Star1})'
     cursor.execute(sqlMsg)
connection.commit()
cursor.close()

#### reviews to postgreSQL("vitagreen"."Comments")

In [46]:
# reviews.to_csv('comment_transformed.csv')
connection = engine.raw_connection()
cursor = connection.cursor()
for index, row in reviews.iterrows():
     sqlMsg = 'INSERT INTO "vitagreen"."Comments" (index, link, comment, stars) \
          VALUES ({0}, \'{1}\', \'{2}\', {3})'.format(index, row.Link, re.sub(r"\'", "\'\'", row.Comment), row.Stars)
     cursor.execute(sqlMsg)
connection.commit()
cursor.close()

#### keywordsDF to postgreSQL("vitagreen"."keywords")

In [47]:
keywordsDF.to_sql('keywords', con=engine, schema='vitagreen', if_exists='replace')

600

#### keywordsIncludedDF to postgreSQL("vitagreen"."included_keywords")

In [48]:
keywordsIncludedDF.to_sql('included_keywords', con=engine, schema='vitagreen', if_exists='replace')

924